### Mount google drive and import libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
root = '/content/drive/My Drive/CS489 Team 3 Project/Irfan\'s code'

In [3]:
import os

# Regular visualization libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


# Keras and tensorflow
import tensorflow as tf
import tensorflow_hub as hub

from keras.regularizers import l1, l2
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import LeakyReLU
from keras import backend
from tensorflow.keras import optimizers

#sklearn and imblearn modules:
from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import confusion_matrix

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


### Load the similarity matrix

In [6]:
e_All_four = np.load(root + '/e_All_four_0_2.npy')

### Initiate the encoder

In [7]:
# Load the encoder:
g = tf.Graph()
with g.as_default():
  text_input = tf.placeholder(dtype=tf.string, shape=[None])
  embed = hub.Module("https://tfhub.dev/google/universal-sentence-encoder-large/3")
  embedded_text = embed(text_input)
  init_op = tf.group([tf.global_variables_initializer(), tf.tables_initializer()])
g.finalize()

# Initialize session:
session = tf.Session(graph=g)
session.run(init_op)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [8]:
scaler = StandardScaler()

### Load the pretrained model

In [9]:
news_DNN_four = tf.keras.models.load_model(root + '/news_DNN_four')
news_DNN_four.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 100)               51300     
                                                                 
 dropout_4 (Dropout)         (None, 100)               0         
                                                                 
 dense_7 (Dense)             (None, 100)               10100     
                                                                 
 dropout_5 (Dropout)         (None, 100)               0         
                                                                 
 dense_8 (Dense)             (None, 4)                 404       
                                                                 
Total params: 61,804
Trainable params: 61,804
Non-trainable params: 0
_________________________________________________________________


###Polarization Score Function

The mechanism is as follows:
<br>
<br>
Since we have four possible outputs (left, left-center, right, right-center), we will determine the score based on the model confident level of the predicted output.

**Left-center and Right center**


```
score = 50  * model's confident level of the predicted output
```


**Left and Right**


```
score = 50 + (50  * model's confident level of the predicted output)
```


In [14]:
def polarization_score(pred_list, cls):
  if cls == 0 or cls == 3:
    return int(50 + 50 * pred_list[cls])
  else:
    return int(50 * pred_list[cls])

### Make Prediction Function

In [17]:
def make_pred(news_text):
  sim_matrix = session.run(embedded_text, feed_dict={text_input: [news_text]})
  combine = np.vstack([e_All_four, sim_matrix])
  sca = scaler.fit_transform(combine)[-1]
  sca = np.array([sca])
  # sca = sca.reshape(sca.shape[0], 1, sca.shape[1])
  pred = news_DNN_four.predict(sca)
  classes_y= np.argmax(pred,axis=1)

  if classes_y[0] == 0:
    return 'Left', polarization_score(pred[0], classes_y)
  elif  classes_y[0] == 1:
    return 'Left-center', polarization_score(pred[0], classes_y)
  elif  classes_y[0] == 2:
    return 'right-center', polarization_score(pred[0], classes_y)
  else:
    return 'right', polarization_score(pred[0], classes_y)

###Prediction example

In [18]:
text = 'Former Vice President Joe Biden (D) defeated Trump in the 2020 presidential election on November 3, 2020. Biden received 306 electoral votes, while Trump received 232. Trump won the 2016 presidential election against Hillary Clinton (D), receiving 304 electoral votes. Key policy initiatives during Trump\'s first term included the United States withdrawing from the Iran nuclear deal, the passage of the Tax Cuts and Jobs Act of 2017 and First Step Act in 2018, reducing the size of the ISIS caliphate, and establishing the Space Force as an independent military branch.[1][2] Trump made more than 200 federal judicial appointments, including three U.S. Supreme Court justices. During his presidency, Trump issued nine vetoes. To read more about these vetoes, click here. On December 18, 2019, Trump became the third president in U.S. history to be impeached by the U.S. House.[3] The U.S. Senate acquitted Trump on both charges of abuse of power and obstruction of Congress on February 5, 2020.[4] Trump was impeached a second time on January 13, 2021, for incitement of insurrection. The Senate acquitted Trump on February 13, 2021.'

make_pred(text)

('Left-center', 25)